In [159]:
using Zygote
function tbf90(I01p, T, incAng, k, a, g, n, eps, dz)
    cos1 = cos(incAng / 180.0 * π)
    Jup0 = zeros(Float64, n + 1)
    Jd0 = zeros(Float64, n + 1)
    Jup = (Jup0)
    Jd = (Jd0)
    intDo0 = zeros(Float64, n + 1)
    intUp0 = zeros(Float64, n + 1)
    intDo = (intDo0)
    intUp = (intUp0)

    for k1 in 1:n+1
        Jup[k1] = (1 - a[k1]) * T[k1] + a[k1] * (0.5 * (I01p[2*k1-1] + I01p[2*k1+1]) * g[k1] * cos1 + I01p[2*k1])
        Jd[k1] = (1 - a[k1]) * T[k1] + a[k1] * (-0.5 * (I01p[2*k1-1] + I01p[2*k1+1]) * g[k1] * cos1 + I01p[2*k1])
    end

    intDo[1] = k[1] * dz / 2
    intUp[n+1] = k[n+1] * dz / 2

    for k1 in 1:n
        intUp[n+1-k1] = intUp[n-k1+2] + 0.5 * (k[n+1-k1] + k[n+1-k1+1]) * dz
    end

    for k1 in 2:n+1
        intDo[k1] = intDo[k1-1] + 0.5 * (k[k1] + k[k1-1]) * dz
    end
    sumJD = Float64(0)
    sumJU = Float64(0)
    #sumJD = 
    #sumJU = Zygote.Buffer(sumJU0)

    for i in 1:n+1
        sumJD += Jd[i] * exp(-intDo[i] / cos1) * k[i] * dz / cos1
        sumJU += Jup[i] * exp(-intUp[i] / cos1) * k[i] * dz / cos1
    end

    Tb = (1 - eps) * exp(-intUp[1] / cos1) * sumJD + sumJU + eps * exp(-intUp[1] / cos1) * T[1]
    return Tb
end

function SetEddington1D(T, k, a, g, n, eps, dz, Ts)
    Abig0 = zeros(Float64, 2*n+3, 2*n+3)
    B0 = zeros(Float64, 2*n+3)
    Abig = Zygote.Buffer(Abig0)
    B = Zygote.Buffer(B0)
 
    for i in 0:n
        Abig[2*i+2, 2*i+2] = 3 * k[i+1] * (1 - a[i+1]) * dz
        Abig[2*i+2, 2*i+3] = 1
        Abig[2*i+2, 2*i+1] = -1
        B[2*i+2] = 3 * k[i+1] * (1 - a[i+1]) * T[i+1] * dz

        if 2*i+3 < 2*n+3
            km = 0.5 * (k[i+1] + k[i+2])
            am = 0.5 * (a[i+1] + a[i+2])
            gm = 0.5 * (g[i+1] + g[i+2])
            Abig[2*i+3, 2*i+3] = km * (1 - am * gm) * dz
            Abig[2*i+3, 2*i+4] = 1
            Abig[2*i+3, 2*i+2] = -1
            B[2*i+3] = 0.0
        end
    end

    Abig[2*n+3, 2*n+2] = 1.0
    Abig[2*n+3, 2*n+3] = -1.0 / 3.0
    Abig[2*n+3, 2*n+1] = -1.0 / 3.0
    B[2*n+3] = 2.7

    Abig[1, 1] = (2 - eps) / (3 * eps)
    Abig[1, 2] = 1.0
    Abig[1, 3] = (2 - eps) / (3 * eps)
    B[1] = Ts
    Abig=copy(Abig)
    B=copy(B)
    return Abig, B 
end


function SetEddington1D_nobuffer(T, k, a, g, n, eps, dz, Ts)
    #Abig = Zygote.Buffer(zeros(Float64, 2*n+3, 2*n+3))
    Abig0 = zeros(Float64, 2*n+3, 2*n+3)
    B0 = zeros(Float64, 2*n+3)
    Abig = (Abig0)
    B = (B0)
    #Zygote.Buffer()
    for i in 0:n
        Abig[2*i+2, 2*i+2] = 3 * k[i+1] * (1 - a[i+1]) * dz
        Abig[2*i+2, 2*i+3] = 1
        Abig[2*i+2, 2*i+1] = -1
        B[2*i+2] = 3 * k[i+1] * (1 - a[i+1]) * T[i+1] * dz

        if 2*i+3 < 2*n+3
            km = 0.5 * (k[i+1] + k[i+2])
            am = 0.5 * (a[i+1] + a[i+2])
            gm = 0.5 * (g[i+1] + g[i+2])
            Abig[2*i+3, 2*i+3] = km * (1 - am * gm) * dz
            Abig[2*i+3, 2*i+4] = 1
            Abig[2*i+3, 2*i+2] = -1
            B[2*i+3] = 0.0
        end
    end

    Abig[2*n+3, 2*n+2] = 1.0
    Abig[2*n+3, 2*n+3] = -1.0 / 3.0
    Abig[2*n+3, 2*n+1] = -1.0 / 3.0
    B[2*n+3] = 2.7

    Abig[1, 1] = (2 - eps) / (3 * eps)
    Abig[1, 2] = 1.0
    Abig[1, 3] = (2 - eps) / (3 * eps)
    B[1] = Ts
    return copy(Abig), copy(B)
end


SetEddington1D_nobuffer (generic function with 1 method)

In [35]:
using Zygote
kext1 = Float64[0.5, 0.6, 0.7, 0.8]
scat1 = Float64[0.3, 0.4, 0.5, 0.6]
asym1 = Float64[0.1, 0.2, 0.3, 0.4]
temp1 = Float64[300.0, 310.0, 320.0, 330.0,330.0]
dz = Float64(0.25)
eps = Float64(0.9)
n = length(kext1) - 1
incAng = Float64(45.0)



45.0

In [105]:
#tb_forward=tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
function tbwrapper_nobuffer(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
    temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
    A,B=SetEddington1D_nobuffer(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
    I01p=A\B;
    Tb_jl=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
    return Tb_jl
end

function tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
    temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
    A,B=SetEddington1D(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
    I01p=A\B;
    Tb_jl=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
    return Tb_jl
end

tbwrapper (generic function with 1 method)

In [147]:
tb_forward=tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
#@time grad = Zygote.gradient((kext1, scat1, asym1, temp1, eps) -> tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng), kext1, scat1, asym1, temp1, eps);



1920.143929510474

In [31]:
#incAng, k, a, g, n, eps, dz
#calculate of the gradient of Tb with respect to kext1, scat1, asym1, temp1, eps, I01p
temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
A,B=SetEddington1D(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
I01p=A\B;
Tb_jl=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
println(typeof(I01p))
println(typeof(temp1))
println(typeof(incAng))
println(typeof(kext1))
println(typeof(scat1))
println(typeof(asym1))
println(typeof(eps))
grad = Zygote.gradient((I01p, temp1, kext1, scat1, asym1, eps) -> tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz), I01p, temp1, kext1, scat1, asym1, eps)

Vector{Float64}
Vector{Float64}
Float64
Vector{Float64}
Vector{Float64}
Vector{Float64}
Float64


([0.0007421181520280422, 0.025220422555734454, 0.00372028866073439, 0.04768987514482673, 0.01137589400917483, 0.08563218592155354, 0.028752316454698786, 0.15072216100757765, 0.020354592954230307], [0.4509583908793771, 0.07153481271724008, 0.08563218592155354, 0.10048144067171845, 0.0], [18.81322314838421, -3.9899222007523854, -5.860094151808049, -9.127123189671082], [-6.8641281521109505, -11.523337430528308, -19.16202519308969, -31.871893045153705], [1.6030832475467638, 4.057293728796677, 9.630742497056513, 21.77468111473663], 68.54896327058623)

In [63]:
using PyCall

In [148]:
# add the current directory to the python path
pushfirst!(PyVector(pyimport("sys")["path"]), "")
read_rte_data=pyimport("read_rte_data")
kext, scat, asym, temp, tb_eddgn=read_rte_data.read_data();

@time for i in 1:1000
    kext1=Float64.(kext[i,:])
    scat1=Float64.(scat[i,:])
    asym1=Float64.(asym[i,:])
    temp1=Float64.(temp[i,:])
    temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
    n = length(kext1) - 1
    A,B=SetEddington1D_nobuffer(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
    #print("$i ")
    I01p=A\B;
    Tb_jl=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
    #grad = Zygote.gradient((kext1, scat1, asym1, temp1, eps) -> tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng), kext1, scat1, asym1, temp1, eps);
end

  0.484004 seconds (50.87 k allocations: 613.447 MiB, 2.71% gc time)


In [129]:
i=1
kext1=Float64.(kext[i,:])
scat1=Float64.(scat[i,:])
asym1=Float64.(asym[i,:])
temp1=Float64.(temp[i,:])
temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
n = length(kext1) - 1
A,B=SetEddington1D_nobuffer(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
Ab,Bb=SetEddington1D(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
#tb_jl=tbwrapper_nobuffer(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
#tb_jl1=tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
I01p=A\B;
I01p1=Ab\Bb;
Tb_jl_2=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
Tb_jl_21=tbf90(I01p1, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
println(Tb_jl_2)
println(Tb_jl_21)

266.0104434246324
484.8569459632107


In [120]:
plt=pyimport("matplotlib.pyplot")
plt.scatter(I01p,I01p1)
plt.show()

In [142]:
function TBF90_B_clean(tb_outb, i01p,  t, incang, k, a, g, n, eps, dz)
    pi = 3.1415
    jd = zeros(Float64, n+1)
    jup = zeros(Float64, n+1)
    cos1 = cos(incang / 180 * pi)
    i01pb=zeros(Float64, 2*n+3)
    tb=zeros(Float64, n+1)
    kb=zeros(Float64, n+1)
    ab=zeros(Float64, n+1)
    gb=zeros(Float64, n+1)
    eps=0.0

    for k1 in 1:n+1
        jup[k1] = (1 - a[k1]) * t[k1] + a[k1] * (0.5 * (i01p[2*k1-1] + i01p[2*k1+1]) * g[k1] * cos1 + i01p[2*k1])
        jd[k1] = (1 - a[k1]) * t[k1] + a[k1] * (-0.5 * (i01p[2*k1-1] + i01p[2*k1+1]) * g[k1] * cos1 + i01p[2*k1])
    end
    
    intdo = zeros(Float64, n+1)
    intup = zeros(Float64, n+1)
    intdo[1] = k[1] * dz / 2
    intup[n+1] = k[n+1] * dz / 2
    
    for k1 in 1:n
        intup[n+1-k1] = intup[n-k1+2] + 0.5 * (k[n+1-k1] + k[n+1-k1+1]) * dz
    end
    
    for k1 in 2:n+1
        intdo[k1] = intdo[k1-1] + 0.5 * (k[k1] + k[k1-1]) * dz
    end
    
    sumjd = 0.0
    for i in 1:n+1
        sumjd += jd[i] * exp(-intdo[i] / cos1) * k[i] * dz / cos1
    end
    
    tb .= 0.0
    intupb = zeros(Float64, n+1)
    temp1 = -(intup[1] / cos1)
    temp = -(intup[1] / cos1)
    tempb = exp(temp1) * tb_outb
    intupb[1] -= (exp(temp1) * (1 - eps) * sumjd / cos1 + exp(temp) * eps * t[1] / cos1) * tb_outb
    sumjub = tb_outb
    tempb1 = exp(temp) * tb_outb
    epsb = t[1] * tempb1 - sumjd * tempb
    tb[1] += eps * tempb1
    sumjdb = (1 - eps) * tempb
    
    kb .= 0.0
    intdob = zeros(Float64, n+1)
    jupb = zeros(Float64, n+1)
    jdb = zeros(Float64, n+1)
    
    for i in n+1:-1:1
        temp1 = -(intup[i] / cos1)
        temp = jup[i] / cos1
        tempb = exp(temp1) * dz * sumjub
        intupb[i] -= exp(temp1) * temp * k[i] * dz * sumjub / cos1
        jupb[i] += k[i] * tempb / cos1
        temp0 = -(intdo[i] / cos1)
        temp1 = jd[i] / cos1
        tempb0 = exp(temp0) * dz * sumjdb
        kb[i] += temp * tempb + temp1 * tempb0
        intdob[i] -= exp(temp0) * temp1 * k[i] * dz * sumjdb / cos1
        jdb[i] += k[i] * tempb0 / cos1
    end
    
    for k1 in n+1:-1:2
        intdob[k1-1] += intdob[k1]
        tempb = dz * 0.5 * intdob[k1]
        intdob[k1] = 0.0
        kb[k1] += tempb
        kb[k1-1] += tempb
    end
    
    for k1 in n:-1:1
        intupb[n-k1+2] += intupb[n+1-k1]
        tempb = dz * 0.5 * intupb[n+1-k1]
        intupb[n+1-k1] = 0.0
        kb[n+1-k1] += tempb
        kb[n+1-k1+1] += tempb
    end
    
    kb[n+1] += dz * intupb[n+1] / 2
    kb[1] += dz * intdob[1] / 2
    gb .= 0.0
    i01pb .= 0.0
    ab .= 0.0
    
    for k1 in n+1:-1:1
        tempb = cos1 * 0.5 * a[k1] * jupb[k1]
        temp = i01p[2*k1-1] + i01p[2*k1+1]
        temp0 = i01p[2*k1-1] + i01p[2*k1+1]
        ab[k1] += (i01p[2*k1] - cos1 * 0.5 * temp0 * g[k1] - t[k1]) * jdb[k1] + (cos1 * 0.5 * temp * g[k1] + i01p[2*k1] - t[k1]) * jupb[k1]
        tb[k1] += (1 - a[k1]) * jdb[k1] + (1 - a[k1]) * jupb[k1]
        i01pb[2*k1] += a[k1] * jdb[k1] + a[k1] * jupb[k1]
        tempb0 = -cos1 * 0.5 * a[k1] * jdb[k1]
        jdb[k1] = 0.0
        i01pb[2*k1-1] += g[k1] * tempb0 + g[k1] * tempb
        i01pb[2*k1+1] += g[k1] * tempb0 + g[k1] * tempb
        gb[k1] += temp0 * tempb0 + temp * tempb
        jupb[k1] = 0.0
    end
    
    tb_outb = 0.0
    return tb, i01pb, kb, ab, gb, epsb
end


TBF90_B_clean (generic function with 1 method)

In [ ]:
function SUML1B_B(lam1, k, a, g, n, t, eps, dz)
    yb = 1.0
    kb = zeros(Float64, n+1)
    tb = zeros(Float64, n+1)
    ab = zeros(Float64, n+1)
    kb .= 0.0
    tb .= 0.0
    ab .= 0.0
    bb = zeros(Float64, 2*n+3)
    
    for i in n:-1:0
        bb[2*i+2] += lam1[2*i+2] * yb
        tempb = dz * 3 * bb[2*i+2]
        bb[2*i+2] = 0.0
        ab[i+1] -= k[i+1] * t[i+1] * tempb
        tempb0 = (1 - a[i+1]) * tempb
        kb[i+1] += t[i+1] * tempb0
        tb[i+1] += k[i+1] * tempb0
    end
    
    yb = 0.0
    return tb, kb, ab
end


In [180]:
tb_jlL=[]
@time for i in 1:1000
    kext1=Float64.(kext[i,:])
    scat1=Float64.(scat[i,:])
    asym1=Float64.(asym[i,:])
    temp1=Float64.(temp[i,:])
    temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
    n = length(kext1) - 1
    eps=0.7
    dz=0.25
    incAng=53.0
    A,b=SetEddington1D_nobuffer(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
    I01p=A\b;
    Tb_jl=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
    tb_outb=1.0

    tb, i01pb, kb, ab, gb, epsb=TBF90_B_clean(tb_outb,I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
    Ainv=inv(A)
    lam1=Ainv*i01pb;
    SUML1B_B(lam1, k, a, g, n, t, eps, dz)
    push!(tb_jlL,Tb_jl)
end


  1.019557 seconds (115.63 k allocations: 902.624 MiB, 2.25% gc time, 4.08% compilation time)


In [178]:
np=pyimport("numpy")
#print(np.corrcoef(tb_eddgn[1:1000],tb_jlL))

PyObject <module 'numpy' from '/Users/mgrecu/miniconda3/lib/python3.9/site-packages/numpy/__init__.py'>

In [179]:

plt.scatter(tb_eddgn[1:1000],tb_jlL)
plt.plot([180,260],[180,260])
plt.show()
